In [1]:
from joblib import dump, load
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [3]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [4]:
low_imp_features = load('/kaggle/input/mchmlmic-new/low_imp_features.joblib')

In [5]:
train = pd.read_parquet('/kaggle/input/mchmlmic-new/train.parquet.gzip')
train = train[~train.index.duplicated(keep='first')]
X = train.drop(pred_cols, axis=1)
y = train[pred_cols]

In [6]:
del(train)
import gc
gc.collect()

0

In [7]:
# drop the features with low importance and return a new dataframe
X = X.drop(columns=low_imp_features)
X

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.079005,0.533688,8.0,8.0,8.0,15.942918,0.001929,179.557310,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.822650,19.838322
AL200800000002,0.090663,0.165650,8.0,8.0,8.0,1.059678,0.001774,179.592150,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.696846,20.007555
AL200800000004,0.085710,0.814697,8.0,8.0,8.0,10.031118,0.001929,179.768220,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.798930,19.863338
AL200800000010,0.105490,8.942595,8.0,8.0,8.0,22.716291,0.001986,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.698520,19.950300
AL200800000011,0.106758,8.940361,8.0,8.0,8.0,41.684960,0.001609,179.742550,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.717968,19.935875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000395,0.007476,0.000000,716.0,716.0,716.0,0.000000,0.000404,-18.019940,51.753593,0.011510,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-17.166506,29.718370
ZW201500000396,0.002424,0.000000,716.0,716.0,716.0,80.522360,0.000402,-17.357134,53.740124,0.011052,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015.0,-17.915289,31.156115
ZW201500000397,0.038481,0.000000,716.0,716.0,716.0,0.298818,0.000336,-16.403381,53.757790,0.010933,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-18.379501,31.872288


In [8]:
test = pd.read_parquet('/kaggle/input/mchmlmic-new/test.parquet.gzip')
test = test.drop(columns=low_imp_features)

In [9]:
model = RandomForestRegressor(n_estimators=3000, max_features=0.9, random_state=42, n_jobs=-1, verbose=1)
model.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 20.5min finished


RandomForestRegressor(max_features=0.9, n_estimators=3000, n_jobs=-1,
                      random_state=42, verbose=1)

In [10]:
# dump(model, '/kaggle/working/test_rf.joblib')

In [11]:
y_pred = model.predict(test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done 3000 out of 3000 | elapsed:    5.0s finished


In [12]:
out = pd.DataFrame(y_pred, columns=pred_cols)
out['DHSID'] = test.index
out = out[['DHSID'] + pred_cols]
out.to_csv('/kaggle/working/new_test.csv', index=False)

In [13]:
out

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,24.738763,24.084873,14.866003,5.521887,91.301917,31.150990
1,AL200800000005,24.932863,24.344977,21.425963,5.331510,86.390430,25.642883
2,AL200800000007,25.182050,25.087753,11.229833,9.896903,78.086400,8.228417
3,AL200800000008,24.957600,24.725107,11.669343,12.236497,67.789027,6.145353
4,AL200800000009,24.902820,24.719190,12.255033,13.144743,69.714983,5.021323
...,...,...,...,...,...,...,...
14995,ZW201500000382,23.860337,23.048507,16.711220,6.738427,73.318987,28.829287
14996,ZW201500000383,23.708047,23.061643,18.719553,5.760030,79.430503,26.089643
14997,ZW201500000386,25.697350,25.152673,25.849883,5.403750,84.410240,25.873757
14998,ZW201500000390,25.981090,24.981043,8.199360,6.066047,94.428503,19.615770
